In [9]:
# -------------------------------------------------
# 0. 라이브러리
# -------------------------------------------------
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

In [10]:
# -------------------------------------------------
# 1. 데이터 로드
# -------------------------------------------------
train      = pd.read_csv("C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/train.csv")              # 학습 데이터
test       = pd.read_csv("C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/test.csv")               # 테스트 데이터
sample_sub = pd.read_csv("C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/sample_submission.csv")  # id 포함 템플릿

LABEL_COL = "fraud"   # 정답 컬럼
ID_COL    = "id"      # ID   컬럼

In [11]:
# -------------------------------------------------
# 2. 단순 전처리
# -------------------------------------------------
feature_cols = [c for c in train.columns if c not in [LABEL_COL, ID_COL]]

# 수치형·범주형 구분
num_cols = train[feature_cols].select_dtypes(include=["number"]).columns
cat_cols = [c for c in feature_cols if c not in num_cols]

# (1) 결측치 채우기
train[num_cols] = train[num_cols].fillna(train[num_cols].median())
test[num_cols]  = test[num_cols].fillna(train[num_cols].median())

for c in cat_cols:
    mode_val = train[c].mode().iloc[0]
    train[c] = train[c].fillna(mode_val)
    test[c]  = test[c].fillna(mode_val)

# (2) 범주형 원‑핫 인코딩
train_enc = pd.get_dummies(train[feature_cols], drop_first=True)
test_enc  = pd.get_dummies(test[feature_cols],  drop_first=True)

# train / test 컬럼 맞추기
test_enc = test_enc.reindex(columns=train_enc.columns, fill_value=0)

X, y = train_enc, train[LABEL_COL]

In [12]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

# -------------------------------------------------
# 3. 검증 (80 : 20)
# -------------------------------------------------
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42,
    n_jobs=-1
)

model.fit(X_tr, y_tr)

val_probs = model.predict_proba(X_val)[:, 1]
val_ap = average_precision_score(y_val, val_probs)
print(f"Validation Average Precision : {val_ap:.6f}")


[LightGBM] [Info] Number of positive: 6216, number of negative: 553784
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069205 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3227
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011100 -> initscore=-4.489648
[LightGBM] [Info] Start training from score -4.489648
Validation Average Precision : 0.163670


In [13]:
# -------------------------------------------------
# 4. 전체 학습 후 제출 파일 생성
# -------------------------------------------------
model.fit(X, y)

test_probs = model.predict_proba(test_enc)[:, 1]
submission = sample_sub.copy()
submission[LABEL_COL] = test_probs
submission.to_csv("C:/Users/zzaer/OneDrive/바탕 화면/2025 채린/아러다/submission.csv", index=False)
print("submission.csv 저장 완료")


[LightGBM] [Info] Number of positive: 7770, number of negative: 692230
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3228
[LightGBM] [Info] Number of data points in the train set: 700000, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011100 -> initscore=-4.489648
[LightGBM] [Info] Start training from score -4.489648
submission.csv 저장 완료
